<a href="https://colab.research.google.com/github/CristianCosci/AGV-Project/blob/refactoring_mobilenet/AGV_emotions_attack/test_script/prove_AGV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b refactoring_mobilenet https://github.com/CristianCosci/AGV-Project.git

!pip install noise

Cloning into 'AGV-Project'...
remote: Enumerating objects: 750, done.
remote: Counting objects: 100% (750/750), done.
remote: Compressing objects: 100% (516/516), done.
remote: Total 750 (delta 195), reused 683 (delta 144), pack-reused 0
Receiving objects: 100% (750/750), 32.90 MiB | 42.16 MiB/s, done.
Resolving deltas: 100% (195/195), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.0 MB/s 
  Created wheel for noise: filename=noise-1.2.2-cp37-cp37m-linux_x86_64.whl size=64372 sha256=0612cf218de1074a6cb7616c883db063c2cd6587be0df7ab3f2689d438e10ff2
  Stored in directory: /root/.cache/pip/wheels/50/4f/1d/3e94460751c993553ba1a52b8e571ba6510701b4d0a68ffeea
Successfully built noise


In [2]:
import sys, os
os.chdir('/content/AGV-Project/AGV_emotions_attack/attacks_emotions/agv/')

import random
import argparse
#set modules path
sys.path.append(os.path.dirname('attack.py'))
sys.path.append(os.path.join(os.path.dirname('attack.py'),'..'))
sys.path.append(os.path.join(os.path.dirname('attack.py'),'..', ".."))
import numpy as np
import pandas as pd

from fitness import inv_attack_rate, inv_attack_rate_multiple
from agv_filters import Filters
from agv_model_loader import ModelLoader
from agv_optimizer import AGVOptimizer
from agv_optimizer import Individual
from agv_datasets import build_model_and_dataset
from agv_datasets import get_model_name_from_dataset
from agv_datasets import database_and_model
from agv_distances import get_distance_functions
from agv_metrics import compute_metricts

from agv_tests import test, test_fits, save_adv_ex  
from agv_tests import mkdir_p, save_adv_best
from log import Log

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils

model, _, _ = build_model_and_dataset('IMAGENET-MOBILENET')

Located Keras-deep-learning-models
images names:  ['65.1.JPEG', '970.2.JPEG', '230.3.JPEG', '809.4.JPEG', '516.5.JPEG', '57.6.JPEG', '334.7.JPEG', '415.8.JPEG', '674.9.JPEG', '332.10.JPEG', '109.11.JPEG', '286.12.JPEG', '370.13.JPEG', '757.14.JPEG', '595.15.JPEG', '147.16.JPEG', '108.17.JPEG', '23.18.JPEG', '478.19.JPEG', '517.20.JPEG', '334.21.JPEG', '173.22.JPEG', '948.23.JPEG', '727.24.JPEG', '23.25.JPEG', '846.26.JPEG', '270.27.JPEG', '167.28.JPEG', '55.29.JPEG', '858.30.JPEG', '324.31.JPEG', '573.32.JPEG', '150.33.JPEG', '981.34.JPEG', '586.35.JPEG', '887.36.JPEG', '32.37.JPEG', '398.38.JPEG', '777.39.JPEG', '74.40.JPEG', '516.41.JPEG', '756.42.JPEG', '129.43.JPEG', '198.44.JPEG', '256.45.JPEG', '725.46.JPEG', '565.47.JPEG', '167.48.JPEG', '717.49.JPEG', '394.50.JPEG', '92.51.JPEG', '29.52.JPEG', '844.53.JPEG', '591.54.JPEG', '358.55.JPEG', '468.56.JPEG', '259.57.JPEG', '994.58.JPEG', '872.59.JPEG', '588.60.JPEG', '474.61.JPEG', '183.62.JPEG', '107.63.JPEG', '46.64.JPEG', '842.65.

In [3]:
os.chdir('/content/AGV-Project/AGV_emotions_attack/')
file_list = os.listdir('attacks_emotions/datasets/images_dataset/ILSVRC2012_img_val_labeled_caffe_200')
origin_class = []
for name in file_list:
  splitted_name = name.split('.')
  origin_class.append(splitted_name[0])

origin_class = [int(i) for i in origin_class]
print(origin_class)

img_path = 'attacks_emotions/datasets/images_dataset/ILSVRC2012_img_val_labeled_caffe_200/'
images = []
for file in file_list:
  img = image.load_img(img_path + file, target_size=(224, 224))
  img_array = image.img_to_array(img)/255
  img_array_expanded_dims = np.expand_dims(img_array, axis=0)
  images.append(img_array_expanded_dims)

images = np.vstack(images)

[970, 369, 366, 595, 334, 183, 352, 473, 236, 107, 65, 705, 147, 771, 573, 757, 558, 159, 646, 516, 227, 646, 751, 196, 504, 103, 934, 771, 175, 449, 935, 788, 474, 934, 58, 627, 394, 975, 74, 28, 977, 392, 802, 590, 632, 949, 645, 858, 230, 528, 270, 343, 277, 383, 483, 92, 981, 878, 173, 686, 872, 70, 57, 101, 21, 230, 498, 548, 201, 780, 167, 257, 852, 212, 994, 289, 970, 666, 108, 686, 330, 906, 29, 846, 841, 553, 476, 324, 875, 415, 674, 32, 424, 467, 461, 180, 687, 583, 809, 586, 844, 159, 22, 638, 398, 517, 23, 46, 158, 872, 129, 270, 349, 109, 50, 122, 781, 637, 842, 867, 328, 887, 620, 703, 777, 493, 544, 948, 248, 396, 969, 198, 150, 887, 272, 937, 591, 259, 373, 717, 486, 115, 173, 80, 142, 468, 265, 213, 334, 870, 763, 311, 608, 55, 328, 361, 825, 457, 370, 478, 436, 487, 332, 718, 726, 272, 565, 39, 588, 993, 647, 756, 276, 390, 283, 256, 664, 163, 16, 167, 516, 720, 584, 339, 659, 789, 23, 479, 149, 959, 358, 160, 638, 727, 937, 286, 725, 118, 275, 140]


In [4]:
predictions = model.predict(images, batch_size=10)
result = imagenet_utils.decode_predictions(predictions)
pred_class = [np.argmax(x) for x in predictions]

id_different_img = []
for i in range(200):
  if pred_class[i] != origin_class[i]:
    print('ELEMENTO: {}, classe originale: {} --- classe predetta: {}'.format(i, origin_class[i], pred_class[i]))
    id_different_img.append(i)

49152/35363 [=========================================] - 0s 0us/step
ELEMENTO: 1, classe originale: 369 --- classe predetta: 379
ELEMENTO: 5, classe originale: 183 --- classe predetta: 221
ELEMENTO: 16, classe originale: 558 --- classe predetta: 212
ELEMENTO: 18, classe originale: 646 --- classe predetta: 798
ELEMENTO: 19, classe originale: 516 --- classe predetta: 431
ELEMENTO: 23, classe originale: 196 --- classe predetta: 192
ELEMENTO: 32, classe originale: 474 --- classe predetta: 735
ELEMENTO: 34, classe originale: 58 --- classe predetta: 77
ELEMENTO: 36, classe originale: 394 --- classe predetta: 389
ELEMENTO: 37, classe originale: 975 --- classe predetta: 977
ELEMENTO: 38, classe originale: 74 --- classe predetta: 78
ELEMENTO: 39, classe originale: 28 --- classe predetta: 119
ELEMENTO: 41, classe originale: 392 --- classe predetta: 115
ELEMENTO: 48, classe originale: 230 --- classe predetta: 231
ELEMENTO: 49, classe originale: 528 --- classe predetta: 709
ELEMENTO: 51, classe o

In [7]:
name_file_different_img = []
for i in range(len(id_different_img)):
  name_file_different_img.append(file_list[id_different_img[i]])

print('file con predizione classe errata: {}'.format(name_file_different_img))
print()

order_different_img = []
for name in name_file_different_img:
  splitted_name = name.split('.')
  order_different_img.append(int(splitted_name[1]))

order_different_img.sort()
print('id ordinati dei file con classe differente {}'.format(order_different_img))

file con predizione classe errata: ['369.87.JPEG', '183.62.JPEG', '558.140.JPEG', '646.118.JPEG', '516.41.JPEG', '196.176.JPEG', '474.61.JPEG', '58.84.JPEG', '394.50.JPEG', '975.150.JPEG', '74.40.JPEG', '28.88.JPEG', '392.124.JPEG', '230.3.JPEG', '528.137.JPEG', '343.136.JPEG', '383.92.JPEG', '878.104.JPEG', '173.22.JPEG', '686.114.JPEG', '57.6.JPEG', '230.180.JPEG', '780.94.JPEG', '167.48.JPEG', '970.2.JPEG', '666.168.JPEG', '108.17.JPEG', '841.70.JPEG', '875.146.JPEG', '415.8.JPEG', '32.37.JPEG', '467.71.JPEG', '22.128.JPEG', '23.18.JPEG', '46.64.JPEG', '620.151.JPEG', '703.192.JPEG', '777.39.JPEG', '493.162.JPEG', '969.139.JPEG', '198.44.JPEG', '150.33.JPEG', '259.57.JPEG', '717.49.JPEG', '486.108.JPEG', '115.154.JPEG', '265.147.JPEG', '213.161.JPEG', '311.125.JPEG', '361.181.JPEG', '436.175.JPEG', '332.10.JPEG', '718.121.JPEG', '272.156.JPEG', '664.119.JPEG', '167.28.JPEG', '516.5.JPEG', '479.85.JPEG', '638.184.JPEG', '937.155.JPEG', '725.46.JPEG']

id ordinati dei file con classe 

In [ ]:
from IPython.display import Image
os.chdir('/content/AGV-Project/AGV_emotions_attack/')
Image(filename=img_path+'65.1.JPEG')

In [ ]:
img = image.load_img(img_path + '65.1.JPEG', target_size=(224, 224))
img_array = image.img_to_array(img)/255
img_array_expanded_dims = np.expand_dims(img_array, axis=0)
predictions = model.predict(img_array_expanded_dims)
result = imagenet_utils.decode_predictions(predictions)
print(np.argmax((predictions)))

65


In [8]:
!python attacks_emotions/agv/agv_attack.py -l TEST.txt -o TEST.json -bs 1 -e 10 -pp  "offsprings" -ps "direct" -po "ES"  -np 10 -el true -s pareto -df1 ssim -db "IMAGENET-MOBILENET"  -nf 3 -r true -lf TEST.out

Located Keras-deep-learning-models
images names:  ['65.1.JPEG', '970.2.JPEG', '230.3.JPEG', '809.4.JPEG', '516.5.JPEG', '57.6.JPEG', '334.7.JPEG', '415.8.JPEG', '674.9.JPEG', '332.10.JPEG', '109.11.JPEG', '286.12.JPEG', '370.13.JPEG', '757.14.JPEG', '595.15.JPEG', '147.16.JPEG', '108.17.JPEG', '23.18.JPEG', '478.19.JPEG', '517.20.JPEG', '334.21.JPEG', '173.22.JPEG', '948.23.JPEG', '727.24.JPEG', '23.25.JPEG', '846.26.JPEG', '270.27.JPEG', '167.28.JPEG', '55.29.JPEG', '858.30.JPEG', '324.31.JPEG', '573.32.JPEG', '150.33.JPEG', '981.34.JPEG', '586.35.JPEG', '887.36.JPEG', '32.37.JPEG', '398.38.JPEG', '777.39.JPEG', '74.40.JPEG', '516.41.JPEG', '756.42.JPEG', '129.43.JPEG', '198.44.JPEG', '256.45.JPEG', '725.46.JPEG', '565.47.JPEG', '167.48.JPEG', '717.49.JPEG', '394.50.JPEG', '92.51.JPEG', '29.52.JPEG', '844.53.JPEG', '591.54.JPEG', '358.55.JPEG', '468.56.JPEG', '259.57.JPEG', '994.58.JPEG', '872.59.JPEG', '588.60.JPEG', '474.61.JPEG', '183.62.JPEG', '107.63.JPEG', '46.64.JPEG', '842.65.

In [ ]:
!python attacks_emotions/agv/agv_attack.py -l TEST.txt -o TEST.json -bs 1 -e 1 -pp  "offsprings" -ps "direct" -po "ES"  -np 1 -el true -s pareto -df1 ssim -db "IMAGENET-MOBILENET"  -nf 3 -r true -lf TEST.out -sae 1

In [ ]:
!python attacks_emotions/agv/agv_attack.py -bf TEST/best_jsons -db "IMAGENET-MOBILENET" -sae_best=True -img_id 10